# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age (Number of years customer is in business)
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer (for this marketing agency)
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
    * Churn - 1 customer has churned, 0 customer is still with the agency
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
import sys
sys.path.append('C:/Users/nishita/exercises_udemy/tools/')
from chinmay_tools import *

In [2]:
from pyspark.sql import SparkSession

* Create a spark session

In [3]:
spark1 = SparkSession.builder.appName('logi_proj_1').getOrCreate()

* Load the cSV file

In [4]:
sdf_customers = spark1.read.csv('Logistic_Regression/customer_churn.csv', inferSchema=True, header=True)

In [5]:
sdf_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [6]:
df = sdf_customers.describe().toPandas()

In [7]:
df.transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
Names,900,None,None,Aaron King,Zachary Walsh
Age,900,41.81666666666667,6.127560416916251,22.0,65.0
Total_Purchase,900,10062.82403333334,2408.644531858096,100.0,18026.01
Account_Manager,900,0.4811111111111111,0.4999208935073339,0,1
Years,900,5.27315555555555,1.274449013194616,1.0,9.15
Num_Sites,900,8.587777777777777,1.7648355920350969,3.0,14.0
Location,900,None,None,"00103 Jeffrey Crest Apt. 205 Padillaville, IA ...",Unit 9800 Box 2878 DPO AA 75157
Company,900,None,None,Abbott-Thompson,"Zuniga, Clark and Shaffer"
Churn,900,0.16666666666666666,0.3728852122772358,0,1


* Check for null values

In [8]:
sdf_customers.filter('Names is null OR Age is null OR Total_Purchase is null OR Account_Manager is null '
                     +'OR Years is null OR Num_Sites is null OR Onboard_date is null '
                     +'OR Location is null OR Company is null OR Churn is null').count()

0

In [9]:
sdf_customers.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

'Age', 'Total_Purchase', 'Account_Manager', 'Years', 'Num_Sites'

In [10]:
sdf_customers.count()

900

* Vectorize the numeric columns using VectorAssembler into a 'features' column to be processed by Spark

In [11]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['Age', 'Total_Purchase', 'Account_Manager', 'Years', 'Num_Sites'], outputCol='features')

In [12]:
sdf2 = assembler.transform(sdf_customers)

In [13]:
sdf2.printSchema()
sdf_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)
 |-- features: vector (nullable = true)

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



* Pick the vectorized 'features' column along with the label column ('Churn'), without any null records, as our final data.
* Split that final data into train and test sets

In [14]:
final_data = sdf2.select('features', 'Churn')
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [15]:
train_data.describe().show()
test_data.describe().show()

+-------+-------------------+
|summary|              Churn|
+-------+-------------------+
|  count|                635|
|   mean|0.16377952755905512|
| stddev|0.37036713206489047|
|    min|                  0|
|    max|                  1|
+-------+-------------------+

+-------+-------------------+
|summary|              Churn|
+-------+-------------------+
|  count|                265|
|   mean|0.17358490566037735|
| stddev| 0.3794687990708427|
|    min|                  0|
|    max|                  1|
+-------+-------------------+



* Instantiate a LogisticRegression model

In [16]:
from pyspark.ml.classification import LogisticRegression
lr_churn = LogisticRegression(labelCol='Churn')

* Fit or train the model to our training data

In [18]:
fitted_churn_model = lr_churn.fit(train_data)
type(fitted_churn_model)

pyspark.ml.classification.LogisticRegressionModel

* Just for our curiosity check/explore the summary of the trained model.
* Compare the mean and standarddeviation of the actual lavel ('Churn') and the predicted label ('oprediction') in the "predictions" dataframe in the summary field of the trained model or fitted model

In [19]:
fitted_churn_model.summary.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              Churn|         prediction|
+-------+-------------------+-------------------+
|  count|                635|                635|
|   mean|0.16377952755905512|0.11811023622047244|
| stddev|0.37036713206489047| 0.3229930322310653|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



* It is better to do some evaluation of our trained model.
* We will do ou revaluation matrix against MLlib evaluation and our test data.
* Evaluate the test data

In [21]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [22]:
predictions_and_labels = fitted_churn_model.evaluate(test_data)

In [23]:
type(predictions_and_labels)

pyspark.ml.classification.BinaryLogisticRegressionSummary

In [24]:
predictions_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|Churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[29.0,9617.59,0.0...|    0|[4.33619789368198...|[0.98708284747953...|       0.0|
|[29.0,11274.46,1....|    0|[4.32026483436165...|[0.98687811163184...|       0.0|
|[29.0,13255.05,1....|    0|[3.93216453654270...|[0.98077562177472...|       0.0|
|[30.0,10183.98,1....|    0|[2.76154719759367...|[0.94056218902118...|       0.0|
|[30.0,10744.14,1....|    1|[1.70261087076890...|[0.84587542210017...|       0.0|
|[31.0,5387.75,0.0...|    0|[2.66119093942064...|[0.93469739698469...|       0.0|
|[31.0,8688.21,0.0...|    0|[6.51122643621272...|[0.99851555154657...|       0.0|
|[32.0,5756.12,0.0...|    0|[4.28996319998580...|[0.98647986954864...|       0.0|
|[32.0,6367.22,1.0...|    0|[2.88191149568013...|[0.94694497958478...|       0.0|
|[32.0,9036.27,0

* Use the BinaryClassificationEvaluator to check the area under the curve

* Evaluate our model predications using BinaryClassificationEvaluator to find area under curse.

In [25]:
bin_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Churn')

# BinaryClassificationEvaluator(labelCol='Churn')
# The parameter "rawPredictionCol" picks by default column name as 'rawPRediction', which is wrong here. WHY? Ask Instructor.

In [26]:
# Using 'prediction' column for rawPrediction field (as used by Instructor)
bin_eval.evaluate(predictions_and_labels.predictions)
printHighlighted('NOTE: a value of 0.5 is equivalent to doing a "random guess" \nHere our result is not bad but not that great as well.')

0.8055390113162597

NOTE: a value of 0.5 is equivalent to doing a "random guess" 
Here our result is not bad but not that great as well.


### Now predict on a new dataset

* To predict on a new data we will train our model on the entire historical data set (not only training data)

In [27]:
final_fitted_model = lr_churn.fit(final_data)

In [28]:
sdf_newcust = spark1.read.csv('Logistic_Regression/new_customers.csv', inferSchema=True, header=True)

In [29]:
sdf_newcust.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|         Company|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|        King Ltd|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:54|21083 Nicole Junc...|   Cannon-Benson|
|  Jeremy Chang|65.0|         100.0|              1|  1.0|     15.0|2006-12-11 07:48:13|085 Austin Views ...|Barron-Robertson|
|Megan Ferguson|32.0|        6487.5|              0|  9.4|     14.0|2016-10-28 05:32:13|922 Wright Branch...|   Sexton-Golden|
|  Taylor Young|32.0|      13147.71|              1| 10.0|      8.0|2012-03-20 00:36:46|Unit 0789 Box 073...|  

* Vectorize the numeric columns into a 'features' column
* Confirm that the results has a 'features' column

In [30]:
sdf_test_newcust = assembler.transform(sdf_newcust)
sdf_test_newcust.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



* Then instead of earlier test data we will evaluate on the new data.
* We will now predict the 'Churn' using transform() call to the all_data fitted model, this adds a 'prediction' column to the result. In fact it adds the 3 columns rasPredition, probability and Churn.

In [31]:
final_results = final_fitted_model.transform(sdf_test_newcust)

In [32]:
final_results.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [33]:
final_results.select('Names', 'prediction').show()

+--------------+----------+
|         Names|prediction|
+--------------+----------+
| Andrew Mccall|       0.0|
|Michele Wright|       1.0|
|  Jeremy Chang|       1.0|
|Megan Ferguson|       1.0|
|  Taylor Young|       0.0|
| Jessica Drake|       1.0|
+--------------+----------+

